# Аналитика бронирования отелей

## Описание полей:

* Hotel – тип отеля (City Hotel или Resort Hotel)  
* Is canceled – бронирование было отменено (1) или нет (0); не отмененное считается успешным
* Lead time – количество дней, прошедших между датой бронирования и датой прибытия  
* Arrival full date – полная дата прибытия
* Arrival date year – год прибытия  
* Arrival date month – месяц прибытия  
* Arrival date week number – номер недели прибытия
* Arrival date day of month – день прибытия
* Stays in weekend nights – количество выходных (суббота или воскресенье), которые гость забронировал для проживания в отеле
* Stays in week nights – количество дней (с понедельника по пятницу), которые гость забронировал для проживания в отеле
* Stays total nights – общее число забронированных ночей (сумма двух предыдущих колонок)
* Adults – число взрослых
* Children – число детей
* Babies – число младенцев
* Meal – выбранный тип питания
* Country – страна происхождения клиента
* Reserved room type – тип зарезервированного номера
* Assigned room type – тип полученного номера (может отличаться от забронированного)
* Customer type – тип бронирования
* Reservation status – значение последнего статуса брони: Canceled — было отменено клиентом; Check-Out — клиент зарегистрировался, но уже покинул отель; No-Show — клиент не зарегистрировался и сообщил администрации отеля причину
* Reservation status date – дата обновления статуса


## Цель: 
 1. Анализ данных о бронировании отелей;
 2. Получение инсайтов для бизнес-решений.

## Задачи:
1. Определить топ стран по количеству бронирований;
2. Анализ сезонности бронирования отелей;
3. Выявление более популярного типа отелей (City Hotel или Resort Hotel);
4. Выявление зависимости между наличием детей у клиентов и частотой отмены бронирования (рассчитать Churn rate - коэффициент оттока).

In [2]:
import pandas as pd
bookings=pd.read_csv('2_bookings.csv', sep=';')



### Предварительный просмотр данных.

In [3]:
bookings.head()



,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


5. Определите название страны из 19-й сверху строки в колонке `Country`

In [4]:
bookings.head(19)['Country']



0     PRT
1     PRT
2     GBR
3     GBR
4     GBR
5     GBR
6     PRT
7     PRT
8     PRT
9     PRT
10    PRT
11    PRT
12    USA
13    ESP
14    PRT
15    IRL
16    PRT
17    IRL
18    FRA
Name: Country, dtype: object

### Количество строк и столбцов датафрейма.

In [5]:
print('Количество строк:', bookings.shape[0])
print('Количество столбцов:', bookings.shape[1])


Количество строк: 119390
Количество столбцов: 21


### Проверка типов данных в столбцах датафрейма и определение преобладающего типа данных.

(Преобладают типы данных int64 и object (по 10 записей))

In [6]:
bookings.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Hotel                      119390 non-null  object 
 1   Is Canceled                119390 non-null  int64  
 2   Lead Time                  119390 non-null  int64  
 3   arrival full date          119390 non-null  object 
 4   Arrival Date Year          119390 non-null  int64  
 5   Arrival Date Month         119390 non-null  object 
 6   Arrival Date Week Number   119390 non-null  int64  
 7   Arrival Date Day of Month  119390 non-null  int64  
 8   Stays in Weekend nights    119390 non-null  int64  
 9   Stays in week nights       119390 non-null  int64  
 10  stays total nights         119390 non-null  int64  
 11  Adults                     119390 non-null  int64  
 12  Children                   119386 non-null  float64
 13  Babies                     11

### Приводим названия столбцов к удобному формату.

Приводим названия столбцов к нижнему регистру и заменяем пробелы на знак нижнего подчеркивания.
Например: `Is Canceled` -> `is_canceled`

Используем получившийся код для всех названий колонок. Пройдя циклом по списку колонок датафрейма, собираем словарь, в котором ключами будут старые названия колонок, а значениями — новые.

In [8]:
dict_bookings = {}
for i in bookings.columns:
    dict_bookings[i] = i.replace(' ', '_').lower()

bookings = bookings.rename(columns=dict_bookings)


### Проверка переименования.
Убеждаемся, что названия столбцов были успешно изменены, выведя обновлённый список названий столбцов.

In [11]:
dict_bookings


{'hotel': 'hotel',
 'is_canceled': 'is_canceled',
 'lead_time': 'lead_time',
 'arrival_full_date': 'arrival_full_date',
 'arrival_date_year': 'arrival_date_year',
 'arrival_date_month': 'arrival_date_month',
 'arrival_date_week_number': 'arrival_date_week_number',
 'arrival_date_day_of_month': 'arrival_date_day_of_month',
 'stays_in_weekend_nights': 'stays_in_weekend_nights',
 'stays_in_week_nights': 'stays_in_week_nights',
 'stays_total_nights': 'stays_total_nights',
 'adults': 'adults',
 'children': 'children',
 'babies': 'babies',
 'meal': 'meal',
 'country': 'country',
 'reserved_room_type': 'reserved_room_type',
 'assigned_room_type': 'assigned_room_type',
 'customer_type': 'customer_type',
 'reservation_status': 'reservation_status',
 'reservation_status_date': 'reservation_status_date'}

In [14]:
bookings = bookings.rename(columns=bookings_dictionary)


NameError: name 'bookings_dictionary' is not defined

In [13]:
bookings.columns




Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

### Пользователи из каких топ-5 стран совершили наибольшее число успешных бронирований?
Бронирование считается успешным, если в дальнейшем не было отменено (значение в колонке is_canceled равно 0).



(Португалия, Великобритания)

In [18]:
bookings.query('is_canceled==0').country.value_counts()


PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
       ...  
AIA        1
SMR        1
GUY        1
TJK        1
PYF        1
Name: country, Length: 165, dtype: int64

In [31]:
bookings.head()



,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


### На сколько ночей (колонка stays_total_nights) в среднем бронируют отели City Hotel и на сколько Resort Hotel?

В нашей сети есть два типа отелей: City Hotel и Resort Hotel. Владельцы сети хотят понимать, отличается ли длительность бронирования в зависимости от типа отеля (см.колонку `hotel`).



(В среднем City Hotel бронируют на 2.98 ночей, Resort Hotel на 4.32 ночей)

In [19]:
bookings.groupby('hotel').stays_total_nights.mean().round(2)



hotel
City Hotel      2.98
Resort Hotel    4.32
Name: stays_total_nights, dtype: float64

### Количество овербукингов в датасете

Бывает, что в отелях случается овербукинг. Тогда тип номера, присвоенного клиенту (см.колонку `assigned_room_type`), будет отличаться от изначально забронированного (см.колонку `reserved_room_type`). Отмена бронирования также считается.



Для подсчёта количества овербукингов отбираем строки, где значения в колонке `assigned_room_type` не совпадают со значениями в колонке `reserved_room_type`. И считаем количество полученных строк.


(Количество овербукинов в датасете: 14917)

In [21]:
bookings.query('assigned_room_type!=reserved_room_type').head()


,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
12,Resort Hotel,0,68,2015-07-01,2015,July,27,1,0,4,...,2,0.0,0,BB,USA,D,E,Transient,Check-Out,2015-07-05
15,Resort Hotel,0,68,2015-07-01,2015,July,27,1,0,4,...,2,0.0,0,BB,IRL,D,E,Transient,Check-Out,2015-07-05
17,Resort Hotel,0,12,2015-07-01,2015,July,27,1,0,1,...,2,0.0,0,BB,IRL,A,E,Transient,Check-Out,2015-07-02
18,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,1,...,2,0.0,0,BB,FRA,A,G,Transient,Check-Out,2015-07-02


In [22]:
bookings.query('assigned_room_type!=reserved_room_type').shape

(14917, 21)

### Анализ сезонности
Анализируем даты запланированного прибытия (см.колонку `arrival_date_year`) и определяем, на какой месяц чаще всего оформляли бронь в 2016 году. И изменился ли самый популярный месяц в 2017 году.



Для этого группируем данные по годам и считаем частоту встречаемости каждого месяца в каждом году.



(В 2015 году было больше всего бронирований в сентябре; в 2016 году в октябре; в 2017 году в мае)

In [23]:
bookings.groupby('arrival_date_year').arrival_date_month.value_counts()

arrival_date_year  arrival_date_month
2015               September             5114
                   October               4957
                   August                3889
                   December              2920
                   July                  2776
                   November              2340
2016               October               6203
                   May                   5478
                   April                 5428
                   September             5394
                   June                  5292
                   August                5063
                   March                 4824
                   July                  4572
                   November              4454
                   February              3891
                   December              3860
                   January               2248
2017               May                   6313
                   April                 5661
                   June                  5

### Месяц и год, когда бронирования отеля типа City Hotel отменялись чаще всего.

(Бронирования City Hotel отменялись чаще всего в 2015 в сентябре, в 2016 октябре, в 2017 в мае)

In [24]:
bookings.query('is_canceled==1 and hotel=="City Hotel"').head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
40061,City Hotel,1,88,2015-07-01,2015,July,27,1,0,4,...,2,0.0,0,BB,PRT,A,A,Transient,Canceled,2015-07-01
40062,City Hotel,1,65,2015-07-01,2015,July,27,1,0,4,...,1,0.0,0,BB,PRT,A,A,Transient,Canceled,2015-04-30
40063,City Hotel,1,92,2015-07-01,2015,July,27,1,2,4,...,2,0.0,0,BB,PRT,A,A,Transient,Canceled,2015-06-23
40064,City Hotel,1,100,2015-07-02,2015,July,27,2,0,2,...,2,0.0,0,BB,PRT,A,A,Transient,Canceled,2015-04-02
40065,City Hotel,1,79,2015-07-02,2015,July,27,2,0,3,...,2,0.0,0,BB,PRT,A,A,Transient,Canceled,2015-06-25


Группируем отфильтрованные данные по годам и считаем количество записей для каждого месяца внутри года

In [25]:
bookings.groupby('arrival_date_year').arrival_date_month.value_counts()


arrival_date_year  arrival_date_month
2015               September             5114
                   October               4957
                   August                3889
                   December              2920
                   July                  2776
                   November              2340
2016               October               6203
                   May                   5478
                   April                 5428
                   September             5394
                   June                  5292
                   August                5063
                   March                 4824
                   July                  4572
                   November              4454
                   February              3891
                   December              3860
                   January               2248
2017               May                   6313
                   April                 5661
                   June                  5

### Какая возрастная группа преобладает среди клиентов?
Сравнение средних значений колонок `adults`, `children` и `babies`.


(Взрослые)

In [28]:
bookings[['adults', 'children', 'babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

### В каком типе отелей преобладает количество детей?

В наших данных информация о количестве детей записана в две колонки: `children` и `babies`. Нам требуется общее число детей. Для этого создаём колонку `total_kids`, объединив столбцы `children` и `babies`.

Проверяем, для отелей какого типа среднее значение этой переменной оказалось наибольшим.



(Для отелей типа Resort Hotel среднее значение детей больше и равно 0.14)

In [29]:
bookings['total_kids'] = bookings.children + bookings.babies


Группируем данные по типу отеля и считаем для каждой группы среднее количество детей.

In [33]:
bookings.groupby('hotel').agg({'total_kids':'mean'}).round(2)

,total_kids
hotel,
City Hotel,0.10
Resort Hotel,0.14


### Churn Rate

Считаем, сколько клиентов было потеряно из-за неуспешного бронирования. То есть считаем метрику Churn Rate.

**Churn rate** (отток, коэффициент оттока) – это процент подписчиков, которые отписались от канала коммуникации, отказались от услуг сервиса в течение определенного периода времени. Т.е. представляет собой отношение количества ушедших пользователей к общему количеству пользователей, выраженное в процентах. Если Churn rate считают для какой-то определенной группы, то и количество ушедших пользователей, и общее количество пользователей считают только по этой группе.

В нашем случае Churn Rate — это процент клиентов, которые отменили бронирование. Анализируем, как эта метрика связана с наличием детей у клиентов. Считаем Churn Rate дважды: для клиентов с детьми и для клиентов без детей. 





 Churn rate больше у клиентов без детей (37.22), у клиентов без детей Churn rate меньше (34.92).

Создаём колонку `has_kids`, в которой значение True, если клиент при бронировании указал ребёнка (детей), иначе – False.

In [37]:
bookings['has_kids'] = bookings['total_kids'] > 0
bookings

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date,total_kids,has_kids
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0,False
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0,False
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02,0.0,False
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02,0.0,False
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017-08-30,2017,August,35,30,2,5,...,0,BB,BEL,A,A,Transient,Check-Out,2017-09-06,0.0,False
119386,City Hotel,0,102,2017-08-31,2017,August,35,31,2,5,...,0,BB,FRA,E,E,Transient,Check-Out,2017-09-07,0.0,False
119387,City Hotel,0,34,2017-08-31,2017,August,35,31,2,5,...,0,BB,DEU,D,D,Transient,Check-Out,2017-09-07,0.0,False
119388,City Hotel,0,109,2017-08-31,2017,August,35,31,2,5,...,0,BB,GBR,A,A,Transient,Check-Out,2017-09-07,0.0,False


Получаем 4 датафрейма, отобрав только нужные записи для каждого:  
1 — клиенты без детей, которые делали бронирование (не важно, отменяли его потом или нет)  
2 — клиенты без детей, которые отменили бронирование  
3, 4 — аналогичные два датафрейма для клиентов с детьми

In [39]:
# клиенты без детей, которые делали бронирование (не важно, отменяли его потом или нет)
reserved_clients_without_kids = bookings.query('has_kids == False')
#клиенты без детей, которые отменили бронирование
canseled_clients_without_kids = bookings.query('is_canceled == 1 and has_kids == False')
#клиенты с детьми, которые делали бронирование
reserved_clients_with_kids = bookings.query('has_kids == True')
#клиенты с детьми, которые отменили бронирование
canseled_clients_with_kids = bookings.query('is_canceled == 1 and has_kids == True')

In [35]:
bookings['has_kids']=bookings.total_kids.apply(has_kids)
bookings['has_kids'].value_counts()

False    110058
True       9332
Name: has_kids, dtype: int64

Считаем количество записей в каждом из четырех датафреймов, чтобы получить число клиентов с теми или иными параметрами.

In [40]:
count_1 = reserved_clients_without_kids.shape[0]
#клиенты без детей, которые отменили бронирование
count_2 = canseled_clients_without_kids.shape[0]
#клиенты с детьми, которые делали бронирование
count_3 = reserved_clients_with_kids.shape[0]
#клиенты с детьми, которые отменили бронирование
count_4 = canseled_clients_with_kids.shape[0]

print(f"Клиенты без детей, которые делали бронирование (не важно, отменяли его потом или нет): {count_1}")
print(f"Клиенты без детей, которые отменили бронирование: {count_2}")
print(f"Клиенты с детьми, которые делали бронирование: {count_3}")
print(f"Клиенты с детьми, которые отменили бронирование: {count_4}")


Клиенты без детей, которые делали бронирование (не важно, отменяли его потом или нет): 110058
Клиенты без детей, которые отменили бронирование: 40965
Клиенты с детьми, которые делали бронирование: 9332
Клиенты с детьми, которые отменили бронирование: 3259


Считаем Churn Rate клиентов без детей: делим число клиентов без детей, которые отменили бронирование, на общее число клиентов без детей и переводим результат в проценты. По аналогии считаем Churn Rate клиентов с детьми.

In [43]:
churn_rate_without_kids = (count_2 / count_1) * 100
churn_rate_with_kids = (count_4 / count_3) * 100

rounded_churn_rate_without_kids = round(churn_rate_without_kids, 2)
rounded_churn_rate_with_kids = round(churn_rate_with_kids, 2)

print(f"Churn Rate клиентов без детей: {rounded_churn_rate_without_kids}")
print(f"Churn Rate клиентов c детьми: {rounded_churn_rate_with_kids}")

Churn Rate клиентов без детей: 37.22
Churn Rate клиентов c детьми: 34.92


## Итоги

1. Больше всего бронирований совершают клиенты, проживающие в Португалии и Великобритании;
2. Анализ сезонности бронирования отелей: в 2015 году было больше всего бронирований в сентябре; в 2016 году в октябре; в 2017 году в мае;
3. Более популярен тип отелей Resort Hotel. В среднем City Hotel бронируют на 2.98 ночей, Resort Hotel на 4.32 ночей;
4. Змачен больший отток (больший процент отмен бронирования) клиентов, имеющих детей (Churn Rate клиентов без детей: 37.22; Churn Rate клиентов c детьми: 34.92).


В этом проекте произведён анализ данных о бронированиях и получены инсайты для бизнес-решений. Для выполнения проекта понадобились навыки работы с библиотекой pandas, а также навыки переименования колонок, использование метрики Churn Rate.